In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

assert torch.cuda.is_available()

In [3]:
from climode.model_function import Climate_encoder_free_uncertain

device = 'cuda'
lr = .0005

model = Climate_encoder_free_uncertain(
    num_channels=5, 
    const_channels=2, 
    out_types=5,
    method="euler",
    use_att=True,
    use_err=True,
    use_pos=False,
)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 300)

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model

Climate_encoder_free_uncertain(
  (vel_f): Climate_ResNet_2D(
    (layer_cnn): ModuleList(
      (0): Sequential(
        (0): ResidualBlock(
          (activation): LeakyReLU(negative_slope=0.3)
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop): Dropout(p=0.1, inplace=False)
          (shortcut): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
          (norm1): Identity()
          (norm2): Identity()
        )
        (1): ResidualBlock(
          (activation): LeakyReLU(negative_slope=0.3)
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d

In [19]:
past_sample = torch.randn(10, 10, 32, 64).to(device)
const_channels_info = torch.randn(1, 2, 32, 64).to(device)
lat_map = torch.randn(32, 64).to(device)
lon_map = torch.randn(32, 64).to(device)
t = torch.arange(2, 8).float().to(device)
data = torch.randn(10, 1, 5, 32, 64).to(device)
model = model.to(device)

with torch.no_grad():
    model.update_param(
        [
            past_sample,
            const_channels_info,
            lat_map,
            lon_map,
        ]
    )
    mean, std, _ = model(t, data)
    
mean.shape, std.shape

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_30709/1719626360.py", line 18, in <module>
    mean, std, _ = model(t, data)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/root/climode-reproducibility/src/climode/model_function.py", line 456, in forward
    return mean, std, s_final[0 : len(s_final) : 6]
  File "/root/climode-reproducibility/src/climode/model_function.py", line 456, in forward
    return mean, std, s_final[0 : len(s_final) : 6]
  File "_pydevd_bundle\\pydevd_cython.pyx", line 1697, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_p

In [17]:
sum(p.numel() for p in model.parameters()
    
    if p.requires_grad)

2752099